In [4]:
import os
# generate top 1000 for retrieval trec:
data_folder = "../msmarco/"

corpus = {}  # dict in the format: passage_id -> passage. Stores all existing passages
collection_filepath = os.path.join(data_folder, 'collection.tsv')

with open(collection_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        pid = int(pid)
        corpus[pid] = passage

queries = {}  # dict in the format: query_id -> query. Stores all training queries
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
    
with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        qid = int(qid)
        queries[qid] = query

In [9]:
i = 0
with open("../msmarco/yifan_bm25_top1000.psg.2019.trec.trec") as f, \
    open("../msmarco/index_yifan_bm25_top1000.psg.2019.tsv", "w") as fo:
        for line in f:
            qid, _, did, _, _, _ = line.split("\t")
            fo.write(f"{qid}\t{did}\t{queries[int(qid)]}\t{corpus[int(did)]}\n")
            i += 1

In [14]:
from collections import defaultdict
import pytrec_eval
from statistics import mean

VALIDATION_METRIC = 'recip_rank'   #'recip_rank' #'ndcg_cut_10' 

qrel_file = "../msmarco/2020qrels-pass.txt" #"../msmarco/qrels.dev.tsv"

qrels = defaultdict(dict)
with open(qrel_file) as f:
    for line in f:
        try:
            qid, _, did, rel = line.strip().split("\t")
        except:
            qid, _, did, rel = line.strip().split(" ")
        if int(rel) > 0:
            qrels[qid][did] = int(rel)
run = defaultdict(dict)
with open("output_index_queries.2020.labeled.splade_max_156000.psg.tsv") as f:
    for line in f:
        qid, did, score = line.split("\t")
        run[qid][did] = float(score)
        
for VALIDATION_METRIC in ['recip_rank','ndcg_cut_10']:
    for top_k in [5,10,20,100]:
        top_run = defaultdict(dict)
        for q in run:
            docs = sorted(run[q].items(), key=lambda x: -x[1])
            for item in docs[:top_k]:
                top_run[q][item[0]] = item[1]
        trec_eval = pytrec_eval.RelevanceEvaluator(qrels, {VALIDATION_METRIC})
        eval_scores = trec_eval.evaluate(top_run)
        print(VALIDATION_METRIC, top_k, mean([d[VALIDATION_METRIC] for d in eval_scores.values()]))



recip_rank 5 0.9465408805031447
recip_rank 10 0.9465408805031447
recip_rank 20 0.9465408805031447
recip_rank 100 0.9465408805031447
ndcg_cut_10 5 0.5015191336899024
ndcg_cut_10 10 0.6654784780392424
ndcg_cut_10 20 0.6654784780392424
ndcg_cut_10 100 0.6654784780392424


In [2]:
with open("../msmarco/yifan_bm25_top1000.psg.train.trec.trec") as f, \
    open("../msmarco/yifan_bm25_top100.psg.train.tsv", "w") as fo:
        for line in f:
            qid, _, did, _, _, _ = line.split("\t")
            fo.write(f"{qid}\t{did}\t{queries[int(qid)]}\t{corpus[int(did)]}\n")

In [20]:
'''
qrel = dict()
with open("../msmarco/2020qrels-pass.txt") as f:
    for line in f:
        qid, _, did, rel = line.split(" ")
        if int(rel) > 0:
            qrel[qid] = did
            
qset = set()
with open("../msmarco/msmarco-passagetest2020-top1000.tsv") as f, \
    open("../msmarco/queries.2020.tsv", "w") as fo:
    for line in f:
        qid, _, qtext, _ = line.split("\t")
        if qid not in qset and qid in qrel:
            fo.write(f"{qid}\t{qtext}\n")
            qset.add(qid)
            
'''

In [2]:
# NEED TO EVALUATE POSITIVE DOCUMENST NOT IN TOP 50 IN THE DATA
queries = {}  # dict in the format: query_id -> query. Stores all training queries
queries_filepath = os.path.join(data_folder, 'queries.train.tsv')
    
with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        qid = int(qid)
        queries[qid] = query
        


In [5]:
from collections import defaultdict
qrel = dict()
with open("../msmarco/qrels.train.tsv") as f:
    for line in f:
        qid, _, did, rel = line.split("\t")
        if int(rel) > 0:
            qrel[qid] = did
            

In [7]:
fo = open("../msmarco/yifan_bm25_top100.psg.train.trec.trec.rest", 'w')
from collections import defaultdict
run_rerank = defaultdict(dict)

with open("output_index_queries.train.splade_max_156000.psg.train.tsv") as f:
    for line in f:
        qid, did, score = line.split("\t")
        run_rerank[qid][did] = float(score)


with open("../msmarco/yifan_bm25_top100.psg.train.trec.trec") as f:
    for line in f:
        qid, _, did, _, _, _ = line.split("\t")
        if did in run_rerank[qid]:
            continue
        fo.write(line)
        
fo.close()

In [41]:

run = defaultdict(dict)

with open("../msmarco/queries.train.index_splade_num1_marginmse.psg.train.trec") as f:
    for line in f:
        qid, _, did, _, _, _ = line.split("\t")
        run[qid][did] = 1
        

In [42]:
with open("../msmarco/index_queries.train.index_splade_num1_marginmse.positive.psg.train.tsv", "w") as fo:
    for q in qrel:
        if qrel[q] not in run[q]:
            fo.write(f"{q}\t{qrel[q]}\t{queries[int(q)]}\t{corpus[int(qrel[q])]}\n")

In [1]:
fo = open("output_index_train_colbert_scores_num2.tsv", "w")
for file in ["output_index_queries.train.index_splade_num1_marginmse.300000+.tsv",\
             "output_index_queries.train.index_splade_num1_marginmse.220000-300000.tsv",\
             "output_index_queries.train.index_splade_num1_marginmse.0-220000.tsv",\
             "output_index_queries.train.index_splade_num1_marginmse.positive.tsv"
            ]:
    with open(file) as f:
        for line in f:
            fo.write(line)
fo.close()

# generate num2

In [2]:

from collections import defaultdict
qrel = dict()
with open("../msmarco/qrels.train.tsv") as f:
    for line in f:
        qid, _, did, rel = line.split("\t")
        if int(rel) > 0:
            qrel[qid] = did
            
run_retriever = defaultdict(dict)

with open("../msmarco/queries.train.index_splade_num1_marginmse.psg.train.trec") as f:
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        run_retriever[qid][did] = float(rank)
        
run_rerank = defaultdict(dict)

with open("output_index_train_colbert_scores_num2.tsv") as f:
    for line in f:
        qid, did, score = line.split("\t")
        run_rerank[qid][did] = float(score)
        

In [2]:
# bm25
from collections import defaultdict
qrel = dict()
with open("../msmarco/qrels.train.tsv") as f:
    for line in f:
        qid, _, did, rel = line.split("\t")
        if int(rel) > 0:
            qrel[qid] = did
            
run_retriever = defaultdict(dict)

with open("../msmarco/queries.train.splade_max_156000.psg.train.trec.trec") as f:
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        run_retriever[qid][did] = float(rank)
        
run_rerank = defaultdict(dict)

with open("output_index_queries.train.splade_max_156000.psg.train.tsv") as f:
    for line in f:
        qid, did, score = line.split("\t")
        run_rerank[qid][did] = float(score)
        

In [3]:

from collections import defaultdict
qrel = dict()
with open("../msmarco/qrels.train.tsv") as f:
    for line in f:
        qid, _, did, rel = line.split("\t")
        if int(rel) > 0:
            qrel[qid] = did
            
run_retriever = defaultdict(dict)

with open("../msmarco/yifan_bm25_top1000.psg.train.trec.trec") as f:
    for line in f:
        qid, _, did, rank, _, _ = line.split("\t")
        run_retriever[qid][did] = float(rank)
        
run_rerank = defaultdict(dict)

with open("../msmarco/bm25_colbert_ranking_train.tsv") as f:
    for line in f:
        qid, did, score = line.split("\t")
        run_rerank[qid][did] = -float(score)
        

In [1]:
import os 
import gzip
import pickle
data_folder = "../msmarco/"
ce_scores_file = os.path.join(data_folder, 'cross-encoder-ms-marco-MiniLM-L-6-v2-scores.pkl.gz')

with gzip.open(ce_scores_file, 'rb') as fIn:
    ce_scores = pickle.load(fIn)
    

In [6]:
#queries里面的negative是按rerank的top排，x[0]是combine的rank，x[1]是did，x[2]是ce的rank
import json,tqdm

fo = open("training_queries_yifan_bm25_official_colbert.json", "w")
for q in tqdm.tqdm(run_rerank):
    query = dict()

    query['qid'] = int(q)
    query['query'] = queries[int(q)]
    did_list = sorted(run_rerank[q].items(), key = lambda x: -x[1])
    combine = []
    retrie = []
    ce = []
    for idx, pair in enumerate(did_list):
        if pair[0] in run_retriever[q]:
            combine.append(1/(idx + 61) + 1/(run_retriever[q][pair[0]] + 60))
            retrie.append(1/(run_retriever[q][pair[0]] + 60))
        else:
            combine.append(1/(idx + 61) + 1/(1001 + 60))
            retrie.append(1/(1001 + 60))
        
        if int(pair[0]) in ce_scores[int(q)]:
            ce.append(ce_scores[int(q)][int(pair[0])])
        else:
            ce.append(0)
            
    combine_rank = [sorted(combine, key = lambda x: -x).index(x) for x in combine]
    retrie_rank = [sorted(retrie, key = lambda x: -x).index(x) for x in retrie]
    ce_rank = [sorted(ce, key = lambda x: -x).index(x) for x in ce]
    
    query['neg'] = [[x,int(y[0]),z] for x,y,z in zip(combine_rank, did_list, ce_rank)]
    query[ "splade_neg"] = [[x,int(y[0]),z] for x,y,z in zip(retrie_rank, did_list, ce_rank)]
    if qrel[q] not in [x[0] for x in did_list]:
        query['pos'] = [[51, int(qrel[q]), 51]]
        query['splade_pos'] = [[51, int(qrel[q]), 51]]
    else:
        pos_idx = ([x[0] for x in did_list]).index(qrel[q])
        query['pos'] = [query['neg'][pos_idx]]
        query['splade_pos'] = [query['splade_neg'][pos_idx]]
    fo.write(f"{q}\t{json.dumps(query)}\n")
    fo.flush()
    
fo.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502939/502939 [34:18<00:00, 244.35it/s]


In [7]:
import json
i =0 
with open("training_queries_yifan_bm25_official_colbert.json") as f, open("training_queries_yifan_bm25_official_colbert_ceclean.json", 'w') as fo:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        idx = 0
        while idx < len(query['neg']):
            if query['neg'][idx][1] not in ce_scores[query['qid']]:
                query['neg'].pop(idx)
                idx -= 1
            idx += 1
            
        idx = 0
        while idx < len(query['splade_neg']):
            if query['splade_neg'][idx][1] not in ce_scores[query['qid']]:
                query['splade_neg'].pop(idx)
                idx -= 1
            idx += 1
        fo.write(f"{qid}\t{json.dumps(query)}\n")

In [10]:
import json
import os
# NEED TO EVALUATE POSITIVE DOCUMENST NOT IN TOP 50 IN THE DATA
queries = {}  # dict in the format: query_id -> query. Stores all training queries
data_folder = "../msmarco"
queries_filepath = os.path.join(data_folder, 'queries.train.expand.splade_num1_marginmse.labeled.tsv')
    
with open(queries_filepath, 'r', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        qid = int(qid)
        queries[qid] = query
 


with open("training_queries_num3_ceclean.json") as f, open("training_queries_num3_ceclean_prf.json", 'w') as fo:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        if query['qid'] in queries:
            query['query'] = queries[query['qid']]
            fo.write(f"{qid}\t{json.dumps(query)}\n")

In [9]:
len(queries)

502938

In [6]:
#queries里面的negative是按rerank的top排，x[0]是combine的rank，x[1]是did，x[2]是ce的rank
import json,tqdm

fo = open("training_queries_yifan_bm25.json", "w")
for q in tqdm.tqdm(run_rerank):
    query = dict()

    query['qid'] = int(q)
    query['query'] = queries[int(q)]
    did_list = sorted(run_rerank[q].items(), key = lambda x: -x[1])
    combine = []
    retrie = []
    ce = []
    for idx, pair in enumerate(did_list):
        if pair[0] in run_retriever[q]:
            combine.append(1/(idx + 61) + 1/(run_retriever[q][pair[0]] + 60))
            retrie.append(1/(run_retriever[q][pair[0]] + 60))
        else:
            combine.append(1/(idx + 61) + 1/(101 + 60))
            retrie.append(1/(101 + 60))
        
        if int(pair[0]) in ce_scores[int(q)]:
            ce.append(ce_scores[int(q)][int(pair[0])])
        else:
            ce.append(0)
            
    combine_rank = [sorted(combine, key = lambda x: -x).index(x) for x in combine]
    retrie_rank = [sorted(retrie, key = lambda x: -x).index(x) for x in retrie]
    ce_rank = [sorted(ce, key = lambda x: -x).index(x) for x in ce]
    
    query['neg'] = [[x,int(y[0]),z] for x,y,z in zip(combine_rank, did_list, ce_rank)]
    query[ "splade_neg"] = [[x,int(y[0]),z] for x,y,z in zip(retrie_rank, did_list, ce_rank)]
    if qrel[q] not in [x[0] for x in did_list]:
        query['pos'] = [[51, int(qrel[q]), 51]]
        query['splade_pos'] = [[51, int(qrel[q]), 51]]
    else:
        pos_idx = ([x[0] for x in did_list]).index(qrel[q])
        query['pos'] = [query['neg'][pos_idx]]
        query['splade_pos'] = [query['splade_neg'][pos_idx]]
    fo.write(f"{q}\t{json.dumps(query)}\n")
    fo.flush()
    
fo.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502043/502043 [32:33<00:00, 256.98it/s]


In [7]:
import json
i =0 
with open("training_queries_yifan_bm25.json") as f, open("training_queries_yifan_bm25_ceclean.json", 'w') as fo:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        idx = 0
        while idx < len(query['neg']):
            if query['neg'][idx][1] not in ce_scores[query['qid']]:
                query['neg'].pop(idx)
                idx -= 1
            idx += 1
            
        idx = 0
        while idx < len(query['splade_neg']):
            if query['splade_neg'][idx][1] not in ce_scores[query['qid']]:
                query['splade_neg'].pop(idx)
                idx -= 1
            idx += 1
        fo.write(f"{qid}\t{json.dumps(query)}\n")

In [4]:
import json
with open("training_queries_num3_ceclean.json") as f:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        pos_id = query['pos'].pop(0)   #Pop positive and add at end
        if pos_id[1] not in corpus:
            print(pos_id[1])
print("end")

end


In [27]:
import json
i =0 
ce_score_margin = 3
with open("training_queries_splade_max_156000.json") as f, open("training_queries_splade_max_156000_ceclean.json", 'w') as fo:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        pos_min_ce_score = ce_scores[query['qid']][query['pos'][0][1]]
        ce_score_threshold = pos_min_ce_score - ce_score_margin
        
        idx = 0
        while idx < len(query['neg']):
            if query['neg'][idx][1] not in ce_scores[query['qid']]:
                query['neg'].pop(idx)
                idx -= 1
            elif ce_scores[query['qid']][query['neg'][idx][1]] > ce_score_threshold:
                query['neg'].pop(idx)
                idx -= 1
            idx += 1
            
        idx = 0
        while idx < len(query['splade_neg']):
            if query['splade_neg'][idx][1] not in ce_scores[query['qid']]:
                query['splade_neg'].pop(idx)
                idx -= 1
            idx += 1
            
        if len(query['neg']) > 0:
            fo.write(f"{qid}\t{json.dumps(query)}\n")

In [4]:
import json
shuffle_triples = set()
with open("training_queries_yifan_bm25_ceclean.json") as f:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        pos_id = query['pos'][0][1]
        pos_idx = query['pos'][0][0]
        
        for neg in query['neg']:
            neg_id = neg[1]
            neg_idx = neg[0]
        
            shuffle_triples.add((qid, pos_id, neg_id, neg_idx - pos_idx))

In [5]:
with open("../msmarco/")
for n in shuffle_triples:
    

8187955

# generate num2 for ance + splade

In [1]:
from collections import defaultdict
import tqdm

splade_run = defaultdict(dict)

'''
with open("../msmarco/queries.train.splade_max_156000.psg.train.trec.trec") as f:
    for line in tqdm.tqdm(f):
        qid, _, did, rank, _, _ = line.strip().split("\t")
        splade_run[qid][did] = int(rank)
'''
with open("../msmarco/queries.train.index_splade_num1_marginmse.psg.train.trec") as f:
    for line in tqdm.tqdm(f):
        qid, _, did, rank, _, _ = line.strip().split("\t")
        splade_run[qid][did] = int(rank)


25146900it [00:26, 948777.43it/s]


In [2]:

ance_run = defaultdict(dict)
q = None

with open("/home/ec2-user/efs/pyserini/runs/run.msmarco-passage.ance.bf.train.tsv") as f:
    for line in tqdm.tqdm(f):
        qid, did, rank = line.strip().split("\t")
        if int(rank) > 100:
            continue
        ance_run[qid][did] = int(rank)

502939000it [06:24, 1307394.12it/s]


In [7]:
#queries里面的negative是按rerank的top排，x[0]是combine的rank，x[1]是did，x[2]是ce的rank
import json,tqdm

fo = open("training_queries_ance_splade_distill.json", "w")
for q in tqdm.tqdm(ance_run):
    query = dict()

    query['qid'] = int(q)
    query['query'] = queries[int(q)]
    did_list = list(ance_run[q].items())
    
    combine = []
    retrie = []
    ce = []
    for idx, pair in enumerate(did_list):
        if pair[0] in splade_run[q]:
            combine.append(1/(pair[1] + 60) + 1/(splade_run[q][pair[0]] + 60))
            retrie.append(1/(splade_run[q][pair[0]] + 60))
        else:
            combine.append(1/(pair[1] + 60) + 1/(101 + 60))
            retrie.append(1/(101 + 60))
        
        if int(pair[0]) in ce_scores[int(q)]:
            ce.append(ce_scores[int(q)][int(pair[0])])
        else:
            ce.append(0)
            
    for d in set(splade_run[q].keys()) - set(ance_run[q].keys()):
        did_list.append([d, 0])
        combine.append(1/(splade_run[q][d] + 60) + 1/(101 + 60))
        retrie.append(1/(splade_run[q][d]))
        if int(d) in ce_scores[int(q)]:
            ce.append(ce_scores[int(q)][int(d)])
        else:
            ce.append(0)
        
            
    combine_rank = [sorted(combine, key = lambda x: -x).index(x) for x in combine]
    retrie_rank = [sorted(retrie, key = lambda x: -x).index(x) for x in retrie]
    ce_rank = [sorted(ce, key = lambda x: -x).index(x) for x in ce]
    
    query['neg'] = [[x,int(y[0]),z] for x,y,z in zip(combine_rank, did_list, ce_rank)]
    query[ "splade_neg"] = [[x,int(y[0]),z] for x,y,z in zip(retrie_rank, did_list, ce_rank)]
    if qrel[q] not in [x[0] for x in did_list]:
        query['pos'] = [[51, int(qrel[q]), 51]]
        query['splade_pos'] = [[51, int(qrel[q]), 51]]
    else:
        pos_idx = ([x[0] for x in did_list]).index(qrel[q])
        query['pos'] = [query['neg'][pos_idx]]
        query['splade_pos'] = [query['splade_neg'][pos_idx]]
    fo.write(f"{q}\t{json.dumps(query)}\n")
    fo.flush()



fo.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502939/502939 [46:35<00:00, 179.90it/s]


In [2]:
import json
i =0 
#ce_score_margin = 3
with open("training_queries_ance_splade_distill.json") as f, open("training_queries_ance_splade_distill_ceclean.json", 'w') as fo:
    for line in f:
        qid, query = line.split("\t")
        query = json.loads(query)
        pos_min_ce_score = ce_scores[query['qid']][query['pos'][0][1]]
        #ce_score_threshold = pos_min_ce_score - ce_score_margin
        
        idx = 0
        while idx < len(query['neg']):
            if query['neg'][idx][1] not in ce_scores[query['qid']]:
                query['neg'].pop(idx)
                idx -= 1
            #elif ce_scores[query['qid']][query['neg'][idx][1]] > ce_score_threshold:
            #    query['neg'].pop(idx)
            #    idx -= 1
            idx += 1
            
        idx = 0
        while idx < len(query['splade_neg']):
            if query['splade_neg'][idx][1] not in ce_scores[query['qid']]:
                query['splade_neg'].pop(idx)
                idx -= 1
            idx += 1
            
        if len(query['neg']) > 0:
            fo.write(f"{qid}\t{json.dumps(query)}\n")

In [2]:
from collections import defaultdict
run = defaultdict(list)
with open("../msmarco/index_splade_num1_marginmse.psg.dev.trec.trec") as f:
    for line in f:
        qid, _, did, rank, _, _ = line.strip().split("\t")
        if int(rank) <= 100:
            run[qid].append(did)
            
with open("../msmarco/index_splade_num1_marginmse.psg.dev.tsv") as f, open("../msmarco/index_splade_num1_marginmse_top100.psg.dev.tsv", "w") as fo:
    for line in f:
        qid, did, _, _ = line.strip().split("\t")
        if did in run[qid]:
            fo.write(line)

In [3]:
!wc -l ../msmarco/index_splade_num1_marginmse_top100.psg.dev.tsv

698000 ../msmarco/index_splade_num1_marginmse_top100.psg.dev.tsv
